In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline 

In [2]:
import earth_model

## PREM and sesimic wave velocities

P- and S-wave velocities are parameterised in PREM in a similar way to density. For example
$V_P$ can be written:

$$
V_P(r) = \left\{
\begin{array}{ll}
      p_{0,0} + p_{0,1}r + p_{0,2}r^2 + p_{0,3}r^3 & r\leq 1221.5 \; \mathrm{km} \\
      p_{1,0} + p_{1,1}r + p_{1,2}r^2 + p_{1,3}r^3 & 1221.5\leq r\leq 3480.0 \; \mathrm{km}\\
      \vdots & \vdots \\
      p_{12,0} + p_{12,1}r + p_{12,2}r^2 + p_{12,3}r^3 & 6368.0\leq r\leq 6371.0 \; \mathrm{km} \\
\end{array} 
\right.
$$

with $V_S$ written in the same way. However, PREM is both anisotropic (in the upper mantle, for
$6151.0\leq r\leq 6346.6 \; \mathrm{km}$) and anelastic (at all depths). We will ignore the anisotropy
(although adding that may be an interesting thing to do) but we do need to consider anelasticity.

A single value is applied to the bulk, $Q_{\kappa}$ and shear, $Q_{\mu}$, quality factor for each layer. This
is the inverse of the dissipation (e.g. $q_{\kappa} = Q^{-1}_{\kappa}$) and can be used to calculate the
seismic velocities at periods other than 1 s (which is the reference period used in PREM). For a period $T$, 
velocities are given by:

$$V_S(r,T) = V_S(r,1)\left(1-\frac{\ln T}{\pi} q_{\mu}(r)\right)$$

and 

$$V_P(r,T) = V_P(r,1)\left(1-\frac{\ln T}{\pi}\left[
    \left(1 - E\right)q_{\kappa}(r) 
    + Eq_{\mu}(r)\right]\right)$$
    
where $E = \frac{4}{3}\left(\frac{V_S(r,1)}{V_P(r,1)}\right)^2$

In [ ]:
# This implements the PREM density model using 

r_earth = 6371 # km

vp_params = np.array([[11.2622,  0.0000, -6.3640,  0.0000],
                      [11.0487, -4.0362,  4.8023, -13.5732],
                      [15.3891, -5.3181,  5.5242, -2.5514],
                      [24.9520, -40.4673, 51.4832, -26.6419],
                      
                      # BOOOOORD NOW
                           [7.9565, -6.4761,  5.5283, -3.0807],
                           [5.3197, -1.4836,  0.0000,  0.0000],
                           [11.2494, -8.0298,  0.0000,  0.0000],
                           [7.1089, -3.8045,  0.00002,  0.0000],
                           [2.6910,  0.6924,  0.0000,  0.0000],
                           [2.6910,  0.6924,  0.0000,  0.0000],
                           [2.9000,  0.0000,  0.0000,  0.0000],
                           [2.6000,  0.0000,  0.0000,  0.0000],
                           [1.0200,  0.0000,  0.0000,  0.0000]])

vs_params = np.array([[13.0885,  0.0000, -8.8381,  0.0000],
                           [12.5815, -1.2638, -3.6426, -5.5281],
                           [7.9565, -6.4761,  5.5283, -3.0807],
                           [7.9565, -6.4761,  5.5283, -3.0807],
                           [7.9565, -6.4761,  5.5283, -3.0807],
                           [5.3197, -1.4836,  0.0000,  0.0000],
                           [11.2494, -8.0298,  0.0000,  0.0000],
                           [7.1089, -3.8045,  0.00002,  0.0000],
                           [2.6910,  0.6924,  0.0000,  0.0000],
                           [2.6910,  0.6924,  0.0000,  0.0000],
                           [2.9000,  0.0000,  0.0000,  0.0000],
                           [2.6000,  0.0000,  0.0000,  0.0000],
                           [1.0200,  0.0000,  0.0000,  0.0000]])




# Turn range of polynomials from 0 - 1 to 0 - r_earth (makes mass easer)
# and puts density into kg/m^3
density_params[:,0] = density_params[:,0] * 1000
density_params[:,1] = (density_params[:,1] * 1000) / r_earth 
density_params[:,2] = (density_params[:,2] * 1000) / (r_earth**2)
density_params[:,3] = (density_params[:,3] * 1000) / (r_earth**3)